# oc and coc case study

This case study concerns the transient difference absorption measured in the visible spectral region after 530 nm excitation of two articial light harvesters, named orange-calix (oc) and calix-orange-calix (coc), in the solvent toluene. The measurement, the global analysis and the target analysis is based upon the model explained in ([Hippius et al 2007][Hippius2007]).

[Hippius2007]: https://www.few.vu.nl/~ivo/pub/2007/HippiusWilliamsWuerthnerJPCC111_13988_13996.pdf "C. Hippius, I.H.M. van Stokkum, E. Zangrando, R.M. Williams, F. Würthner, Excited state interactions in calix[4]arene-perylene bisimide dye conjugates: Global and target analysis of supramolecular building blocks, Journal of Physical Chemistry C, 111 (2007) 13988-13996."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

oc_tol_data_path = "2016oc_tol.ascii"
plot_data_overview(oc_tol_data_path, nr_of_data_svd_vectors=5, linlog=True, linthresh=1)
coc_tol_data_path = "2016coc_tol.ascii"
plot_data_overview(coc_tol_data_path, nr_of_data_svd_vectors=5, linlog=True, linthresh=1);

## Refined Second Target Analysis: heterogeneous kinetics and small triplet yield

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

oc_coc_tol_target_model_path = "models/oc_coc_tol-target_refined-model.yml"
oc_coc_tol_target_parameters_path = "models/oc_coc_tol-target_refined-parameters.yml"

#### Model file

In [ ]:
display_file(oc_coc_tol_target_model_path, syntax="yaml")

#### Parameters file

Exercise 14: Make sure you thoroughly understand the refined target model that is applied here.

Describe the model qualitatively in your own words.

Hints: solvation, heterogeneous kinetics, **oc vs coc**, triplet yield, spectral assumptions, kinetic assumptions.

Exercise 15: Study the parameters file carefully. Insert the missing starting values in the models/oc_coc_tol-target_refined-parameters.yml file. Hint: use the estimated kinetic and IRF parameters from the 2nd target analysis. Comment on the amount of free parameters. Are they all indispensable?

In [ ]:
display_file(oc_coc_tol_target_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

oc_coc_tol_target_scheme = Scheme(
    model=oc_coc_tol_target_model_path,
    parameters=oc_coc_tol_target_parameters_path,
    maximum_number_function_evaluations=4,
    data={"oc_tol_data": oc_tol_data_path, "coc_tol_data": coc_tol_data_path},
)
oc_coc_tol_target_scheme.validate()

In [ ]:
oc_coc_tol_target_result = optimize(oc_coc_tol_target_scheme)

In [ ]:
oc_coc_tol_target_result

In [ ]:
oc_coc_tol_target_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

result1 = oc_coc_tol_target_result.data["oc_tol_data"]
result2 = oc_coc_tol_target_result.data["coc_tol_data"]
allresults = [result1, result2]
# wavelengths=select_plot_wavelengths(result1,wavelength_range=(420,770))
wavelengths = select_plot_wavelengths(result1)
fig3tr, axes = plot_fitted_traces(allresults, wavelengths, linlog=True, linthresh=1)

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    oc_coc_tol_target_result.data["oc_tol_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

In [ ]:
plot_overview(
    oc_coc_tol_target_result.data["coc_tol_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

## Dispersion curves

In [ ]:
oc_coc_tol_target_result.data["oc_tol_data"].data.sel(time=slice(0, 2)).plot()
oc_coc_tol_target_result.data["oc_tol_data"].irf_center_location.plot()

In [ ]:
oc_coc_tol_target_result.data["coc_tol_data"].data.sel(time=slice(0, 2)).plot()
oc_coc_tol_target_result.data["coc_tol_data"].irf_center_location.plot()

Exercise 16: Compare the quality of the fit of the simultaneous refined 2nd target analysis (summarized by the rms error and the first LSV and RSV) with that of the individual global, 1st and 2nd target analysis.

Exercise 17: Comment on the interpretation of the SADS, with the help of the literature, and relate this to the target model used.

In [ ]:
compartments = oc_coc_tol_target_scheme.model.initial_concentration["input2"].compartments

oc_coc_tol_target_scheme.model.k_matrix["km2fast"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
compartments = oc_coc_tol_target_scheme.model.initial_concentration["input2"].compartments

oc_coc_tol_target_scheme.model.k_matrix["km2slow"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(oc_coc_tol_target_result)